In [1]:
'''
这一块是将 原数据处理成 可以输入到 word2vec 的格式
即一个句子："With all this stuff going down at...."
变成  ['with','all','this'....]

'''

'''加载数据'''
import pandas as pd
# Read data from files 
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

'''为了训练Word2Vec，最好不要去掉停用词，因为算法依赖于更广泛的句子上下文来产生高质量的词向量'''
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

'''处理每一条评论'''
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words, optionally removing stop words.  Returns a list of words.
    # 将文档转换为一系列单词的功能，可选地删除停用词。 返回单词列表。
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 5. Return a list of words
    return(words)

'''
Word2Vec期望单个句子，每个单词作为单词列表。 换句话说，输入格式是一个列表的列表(a list of list)。
使用NLTK的punkt标记器进行句子分割。
'''
import nltk.data
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. 
    # Returns a list of sentences, where each sentence is a list of words
    ''' 将一个review变成几个sentences，每个sentence是 a list of words ,返回一个 list '''
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


'''按照上面将训练集和测试集清洗一下'''
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

'''形成了可用于word2ver的数据'''

C:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [2]:

# Vector Averaging

import numpy as np

def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")

    nwords = 0.
    
    # Index2word is a list that contains the names of the words in the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs


# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

num_features = 300

#形成特征
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )


C:\Anaconda3\lib\site-packages\gensim-3.2.0-py3.6-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Review 0 of 25000


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 220

In [5]:
print(trainDataVecs)
testDataVecs

print(trainDataVecs.shape)
print(testDataVecs.shape)

[[  1.40772052e-02  -5.53002208e-03  -1.08157452e-02 ...,   2.35068612e-03
   -7.21874065e-04   7.29449606e-03]
 [ -2.43310649e-02  -4.59439773e-03  -4.30878205e-03 ...,  -4.86835372e-03
    8.11467133e-03   3.75781138e-03]
 [  5.82704507e-03   2.39496920e-02  -1.99921569e-03 ...,   6.02059532e-03
   -6.38188748e-03   6.21137500e-04]
 ..., 
 [  1.86119117e-02  -9.05444194e-03  -6.79665664e-03 ...,   1.85082015e-02
    1.12793548e-02   2.01262552e-02]
 [ -3.44938301e-02   2.75852121e-02  -2.28196792e-02 ...,  -3.77218379e-03
   -6.31709956e-03   3.89071880e-03]
 [  1.65418989e-03   2.76210415e-03  -1.78317504e-03 ...,  -1.19509771e-02
    8.02366436e-03   3.48768554e-05]]
(25000, 300)
(25000, 300)


In [9]:
# 我把清洗过的训练集和测试集给保存到文件里。
output = pd.DataFrame( data={"id":train["id"], "sentiment":train["sentiment"], "review":clean_train_reviews} )
output.to_csv("data/temp/clean_train_reviews.csv")

output = pd.DataFrame( data={"id":test["id"], "review":clean_test_reviews} )
output.to_csv("data/temp/clean_test_reviews.csv")


Exception: Data must be 1-dimensional

In [10]:
# 我把经过word2ver处理后的数据给保存到文件里。
output = pd.DataFrame( data={"id":train["id"], "sentiment":train["sentiment"], "review":trainDataVecs.tolist()} )
output.to_csv("data/temp/trainDataVecs.csv")

output = pd.DataFrame( data={"id":test["id"], "review":testDataVecs.tolist()} )
output.to_csv("data/temp/testDataVecs.csv")

In [11]:
trainVec = pd.read_csv( "data/temp/trainDataVecs.csv")
trainVec

,Unnamed: 0,id,review,sentiment
0,0,"""5814_8""","[0.014077205210924149, -0.005530022084712982, ...",1
1,1,"""2381_9""","[-0.024331064894795418, -0.004594397731125355,...",1
2,2,"""7759_3""","[0.005827045068144798, 0.023949692025780678, -...",0
3,3,"""3630_4""","[-0.014982522465288639, 0.016666142269968987, ...",0
4,4,"""9495_8""","[0.006841480266302824, 0.015505671501159668, -...",1
5,5,"""8196_8""","[0.03453288599848747, 0.00220335740596056, 0.0...",1
6,6,"""7166_2""","[0.0097458241507411, -0.004650068003684282, 0....",0
7,7,"""10633_1""","[0.009654407389461994, 0.0030965993646532297, ...",0
8,8,"""319_1""","[0.01088715996593237, 0.004521975293755531, -0...",0
9,9,"""8713_10""","[0.03168283775448799, -0.03204600512981415, -0...",1


In [12]:
trainVec = trainVec["review"]
trainVec

0        [0.014077205210924149, -0.005530022084712982, ...
1        [-0.024331064894795418, -0.004594397731125355,...
2        [0.005827045068144798, 0.023949692025780678, -...
3        [-0.014982522465288639, 0.016666142269968987, ...
4        [0.006841480266302824, 0.015505671501159668, -...
5        [0.03453288599848747, 0.00220335740596056, 0.0...
6        [0.0097458241507411, -0.004650068003684282, 0....
7        [0.009654407389461994, 0.0030965993646532297, ...
8        [0.01088715996593237, 0.004521975293755531, -0...
9        [0.03168283775448799, -0.03204600512981415, -0...
10       [-0.011291798204183578, 0.03548548370599747, -...
11       [-0.02834823727607727, -0.007936855778098106, ...
12       [-0.01724749058485031, 0.017894187942147255, -...
13       [0.0039222934283316135, -0.010340132750570774,...
14       [0.0011025972198694944, -0.012920858338475227,...
15       [-0.00928712822496891, 0.005248045548796654, 0...
16       [0.010807695798575878, 0.0034786511678248644, .

In [17]:
trainVec = trainVec.tolist()
trainVec

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
